In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten, Add, InputLayer
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
tf.test.gpu_device_name()

Using TensorFlow backend.


'/device:GPU:0'

In [2]:
ImageHeight = 224
ImageWidth = 224
ArchSelect = 'VGG16'

### Build Structure of Neural Network

In [3]:
if ArchSelect == 'VGG16':
    print('VGG16 Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', input_shape=(ImageHeight, ImageWidth, 1), activation='relu'))
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

elif ArchSelect == 'LeNet':
    print('LeNet Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(5,5), input_shape=(ImageHeight, ImageWidth,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=120))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=84))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

else:
    print('Regular CNN Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', input_shape=(ImageHeight, ImageWidth,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

VGG16 Structure : 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)       29

### Load Train Data

In [4]:
import cv2 as cv
import gc
SampleNumber = 40
train_label = np.zeros(shape=(SampleNumber*5,), dtype = np.float16)


train_down = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('down/down%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_down[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample] = 1

train_up = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('up/up%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_up[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber] = 2

train_right = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('right/right%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_right[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*2] = 3
    
train_left = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('left/left%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_left[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*3] = 4

train_other = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('other/other%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_other[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*4] = 0
    
    
train_set = np.append(train_down, train_up ,axis=0)
train_set = np.append(train_set, train_right ,axis=0)
train_set = np.append(train_set, train_left ,axis=0)
train_set = np.append(train_set, train_other ,axis=0)
del train_down, train_up, train_left, train_right, train_other
gc.collect()

train_labels = np_utils.to_categorical(train_label)

print("train set shape :",train_set.shape)
print("train label shape :",train_labels.shape)
# down : 1, up : 2, right : 3, left : 4, other :0

train set shape : (200, 224, 224, 1)
train label shape : (200, 5)


### make sure train data

In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
plt.figure(figsize=(20,20))
plt.subplot(2,3,1)
img = Image.fromarray(train_set[0].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,2)
img = Image.fromarray(train_set[20].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,3)
img = Image.fromarray(train_set[40].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,4)
img = Image.fromarray(train_set[60].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,5)
img = Image.fromarray(train_set[80].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,6)
img = Image.fromarray(train_set[99].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)

### Training Model

In [6]:
batch_size = 10
epochs = 30

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),metrics=['accuracy'])
history = model.fit(x=train_set, y=train_labels, batch_size=batch_size, epochs=epochs, verbose=1,shuffle=True, validation_split=0.05)

Train on 190 samples, validate on 10 samples
Epoch 1/30
190/190 [==============================] - 18s 94ms/step - loss: 0.9770 - acc: 0.6368 - val_loss: 4.2284 - val_acc: 0.1000
Epoch 2/30
190/190 [==============================] - 10s 51ms/step - loss: 0.6418 - acc: 0.7737 - val_loss: 1.5544 - val_acc: 0.5000
Epoch 3/30
190/190 [==============================] - 10s 52ms/step - loss: 0.5013 - acc: 0.8474 - val_loss: 1.7563 - val_acc: 0.5000
Epoch 4/30
190/190 [==============================] - 10s 51ms/step - loss: 0.5074 - acc: 0.8368 - val_loss: 1.8011 - val_acc: 0.5000
Epoch 5/30
190/190 [==============================] - 9s 48ms/step - loss: 0.4883 - acc: 0.8579 - val_loss: 0.3662 - val_acc: 0.7000
Epoch 6/30
190/190 [==============================] - 10s 50ms/step - loss: 0.4449 - acc: 0.8737 - val_loss: 0.5060 - val_acc: 0.7000
Epoch 7/30
190/190 [==============================] - 10s 50ms/step - loss: 0.3259 - acc: 0.8947 - val_loss: 2.2608 - val_acc: 0.4000
Epoch 8/30
190/190

In [7]:
from keras.models import load_model
model.save('model_DHG_VGG16_224_S40.h5') 